# INTERACTIVE MAPS WITH ITALIAN PARKS

In [1]:
# Creation of the interactive map. Starting source from L. Acconito Thesis.

In [2]:
import folium 
print(folium.__version__)
from folium.plugins import MarkerCluster
import pandas as pd
import branca
from branca.element import MacroElement
from jinja2 import Template
import json

0.14.0


STEP #1: Creiamo base maps with markers

In [3]:
#Creation of the basemap 

m = folium.Map(location = [43,12], zoom_start= 5.3 )  


In [4]:
# Tiles of the map customization
# There are some default tiles by Folium and costum with the website leaflet-extras.github.io

#1
attr = (
    'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
tiles = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
folium.TileLayer(tiles=tiles, attr=attr, name="WorldImagery").add_to(m)

#2
attr = (
    '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
)
tiles = "https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png"
folium.TileLayer(tiles=tiles, attr=attr, name="CartoDB").add_to(m)

#3
folium.TileLayer("OpenStreetMap",show=False).add_to(m)

# piece of code to check if the options are correctly implemented 
# Layer and visualization of the map
# folium.LayerControl().add_to(m)
# m

In [5]:
# Geographical boundaries - GeoJson file
# "show = False" == not shown at the opening of the map
folium.GeoJson("limits_IT_regions.geojson", name = "Regions borders", show=False).add_to(m)
folium.GeoJson("limits_IT_provinces.geojson", name = "Provinces bordes", show=False).add_to(m)
folium.GeoJson("limits_IT_municipalities.geojson", name = "Municipalities borders", show=False).add_to(m)

STEP #2: Colormap base don the EF results of the 5 rated power

In [6]:
# Opne the json file for assessing the region name of the file and store it

In [7]:
with open ('limits_IT_regions.geojson') as f:
    regioni_geojson = json.load(f)
    
regioni_geojson 
for item in regioni_geojson["features"]:
    item['regioni'] = item['properties']['reg_name']
    #print(item['regioni'])


In [8]:
# Read the excel file with tha table: emission factor per rated power, per region -> this average values are the one visualized in the colormap

In [9]:
regioni_ef=pd.read_excel(r'C:\Users\Relab\OneDrive\Desktop\TESI\Eolico\Result\Emission_factor.xlsx',
                    'Regioni_maps', usecols='A:F')

# regioni_ef

In [10]:
# Colormap code

In [11]:
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

STEP #3: Anemometers and Wind farms implementation

In [12]:
# Read the excel file with information on anemometer
anemometer = pd.read_excel(r'C:\Users\Relab\OneDrive\Desktop\PY_Thesis\Impianti_new_version.xlsx',
                    'PositionAnemMap', usecols='A:G')
#anemometer

In [13]:
# Creation of the anemometer icon
anem_group = folium.FeatureGroup(name="Anemometers/epw files", show = False).add_to(m)

for index in range(len(anemometer)-1):
    IconWT = folium.features.CustomIcon('C:/Users/Relab/OneDrive/Desktop/TESI/Eolico/Maps/Picture_anem2.png', icon_size =(20,30))
    iframe = folium.IFrame("<h2><b>"+str(anemometer.at[index,'Anem']) +"</b></h2>" +"<br>"+"<b>Elevation : </b> "+str(anemometer.at[index,'Elevation_anem'])+"m" +"<br>"+"<b>Average velocity : </b> "+str(anemometer.at[index,'V_mean_anem']) + "m/s")
    PopupWT = folium.Popup(iframe, min_width=300, max_width=300)
    anem_group.add_child(folium.Marker(location= [anemometer.at[index,'N'],anemometer.at[index,'E']],
                  popup = PopupWT,
                  tooltip='Anemometer',
                  icon = IconWT
                ))
    
#folium.LayerControl().add_to(m)

In [14]:
# Read the Excel file with wind farms information
Excel=pd.read_excel(r'C:\Users\Relab\OneDrive\Desktop\PY_Thesis\Impianti_new_version.xlsx',
                    'Maps_park_copy', usecols='A:N')
#Excel

In [15]:
########### With this piece of code there is not the clusters with the visualization of the farms grouped when zoom in/out

# Creation of the layer (i) all italian farms (default visualization) 
#WT_total_group = folium.FeatureGroup(name="Parks in ITALY").add_to(m)

#for index in range(len(Excel)-1):
#    IconWT = folium.features.CustomIcon('C:/Users/Relab/OneDrive/Desktop/TESI/Eolico/Maps/Picture_wind.png', icon_size =(20,30))
#    iframe = folium.IFrame("<h2><b>"+str(Excel.at[index,'Location']) +"</b></h2>" +"<br>"+"<b>Number of WT : </b> "+str(Excel.at[index,'N_WT']) +"<br>"+"<b>Model of WT : </b> "+str(Excel.at[index,'Model'])  +"<br>"+"<b>Total power installed : </b> "+str(Excel.at[index,'P_tot']) + "MW"+"<br>"+"<b>Yearly productivity  : </b> "+str(Excel.at[index,'Power_cs_park']) + "MWh"+"<br>"+"<b>Equivalent hours  : </b> "+str(Excel.at[index,'Eq_h']) + "h")
#    PopupWT = folium.Popup(iframe, min_width=300, max_width=300)
#    WT_total_group.add_child(folium.Marker(location= [Excel.at[index,'N'],Excel.at[index,'E']],
#                  popup = PopupWT,
#                  tooltip=[Excel.at[index,'Location']],
#                  icon = IconWT
#                ))


In [16]:
# Creation of the layer (i) all italian farms (default visualization) 
# Create a MarkerCluster for wind turbines
WT_cluster = MarkerCluster(name="Parks in ITALY").add_to(m)

for index in range(len(Excel) - 1):
    # Create a custom icon for wind turbines
    IconWT = folium.features.CustomIcon(
        'C:/Users/Relab/OneDrive/Desktop/TESI/Eolico/Maps/Picture_wind.png',
        icon_size=(20, 30)
    )
    
    # Create a popup for the wind turbine information
    iframe = folium.IFrame(
        f"<h2><b>{Excel.at[index, 'Location']}</b></h2>"
        f"<br><b>Number of WT: </b> {Excel.at[index, 'N_WT']}"
        f"<br><b>Model of WT: </b> {Excel.at[index, 'Model']}"
        f"<br><b>Total power installed: </b> {Excel.at[index, 'P_tot']} MW"
        f"<br><b>Yearly productivity: </b> {Excel.at[index, 'Power_cs_park']} MWh"
        f"<br><b>Equivalent hours: </b> {Excel.at[index, 'Eq_h']} h"
    )
    PopupWT = folium.Popup(iframe, min_width=300, max_width=300)

    # Add each marker to the cluster
    folium.Marker(
        location=[Excel.at[index, 'N'], Excel.at[index, 'E']],
        popup=PopupWT,
        tooltip=Excel.at[index, 'Location'],
        icon=IconWT
    ).add_to(WT_cluster)


In [17]:
# Creation of the layer (ii) region-specific wind farms (w/out the MarkerCluster)
Regions = ['Piemonte','Valle Aosta','Trentino','Veneto','Liguria', 'Emilia Romagna','Marche',
          'Umbria','Toscana','Lazio','Molise','Sardegna','Abruzzo', 'Calabria', 'Basilicata',
          'Sicilia','Puglia','Campania']

for region in Regions:
    Regione_on = list()
    for index in range(len(Excel)-1):
        if Excel.Region[index] ==  region: 
            #entra ogni volta e crea un gruppo ogni volta
            # devo inserire come i punti (location) nella source - https://stackoverflow.com/questions/55344513/add-menu-bar-on-folium-map-to-select-or-deselect-particular-object-marker
            Regione_on.append([Excel.at[index,'N'],Excel.at[index,'E']])
   
    WTs = folium.FeatureGroup(name=f"Parks in {region}", show=False).add_to(m)

    for loc in Regione_on:
        IconWT = folium.features.CustomIcon('C:/Users/Relab/OneDrive/Desktop/TESI/Eolico/Maps/Picture_wind.png', icon_size =(20,30))
        iframe = folium.IFrame("<h2><b>"+str(Excel.at[index,'Location']) +"</b></h2>" +"<br>"+"<b>Number of WT : </b> "+str(Excel.at[index,'N_WT']) +"<br>"+"<b>Model of WT : </b> "+str(Excel.at[index,'Model'])  +"<br>"+"<b>Total power installed : </b> "+str(Excel.at[index,'P_tot']) + "MW"+"<br>"+"<b>Park yearly productivity  : </b> "+str(Excel.at[index,'Power_cs_park']) + "MWh")
        PopupWT = folium.Popup(iframe, min_width=300, max_width=300)
        WTs.add_child(folium.Marker(location= loc,
                      popup = PopupWT,
                      tooltip=[Excel.at[index,'Location']],
                      icon = IconWT
                    ))
        


In [18]:
# add the layers (all the previous option are reported in this layer)    
folium.LayerControl().add_to(m)

In [19]:
# add the legend, bottom left 
m.get_root().html.add_child(folium.Element("""
<div style="position: fixed; 
     bottom: 10px; left: 10px; width: 150px; height: 90px; 
      background-color:#E5E4E2; border:2px solid grey;z-index: 99999; font-size 12px;">
    Legend <br>
    Wind Farm  <i class="fa fa-map-marker fa-2x" style="color:#004386"></i><br>
    Anemometer <i class="fa fa-map-marker fa-2x" style="color:#DB873A"></i><br>
</div>
"""))
#Source
#https://nagasudhir.blogspot.com/2021/08/inject-html-into-python-folium-maps.html
#https://www.datasciencecentral.com/fun-with-maps-part-2/

STEP #4: add title in the map

In [20]:
from branca.element import Element

# Custom top banner HTML and CSS
header_html = """
<style>
    .top-banner {
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 50px;
        background-color: #033E3E;
        color: white;
        font-size: 20px;
        font-weight: bold;
        font-family: 'Aptos', sans-serif;
        line-height: 50px;
        padding-left: 20px;
        z-index: 9999;
        box-shadow: 0 2px 5px rgba(0,0,0,0.3);
    }

    body {
        padding-top: 50px; /* Push the map down below the banner */
    }
</style>

<div class="top-banner">
    Italian Onshore Wind Turbines Database
</div>
"""

# Inject header into the map
header_element = Element(header_html)
m.get_root().html.add_child(header_element)

# Save to HTML
#m.save("wind_map_with_banner.html")

STEP #5: Save the map (.html file)

In [21]:
# .html file saving 
# by default saved in the code folder

m.save("ItalyWindTurbineDb.html") 